## Calculate Phi spatial average within a basin or within LMEs

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]
tres = 'monthly'

In [3]:
datadir = '/glade/campaign/cesm/collections/CESM1-DPLE/DPLE-FOSI/rerun_with_extension/timeseries'
filename = f'{datadir}/g.DPLE.GECOIAF.T62_g16.009.chey.pop.h.TEMP.024901-031812.nc'
dset = xr.open_dataset(filename)
dz = dset['dz'].isel(z_t=range(0, 35))/1e2 # m
darea = dset['TAREA'].isel(nlat=range(a1, a2), nlon=range(b1, b2))/1e10 #km*km
KMT = dset['KMT'].isel(nlat=range(a1, a2), nlon=range(b1, b2))
dset.close()

In [4]:
darea = darea*dz/dz
darea = darea.transpose("z_t","nlat","nlon")

In [5]:
for xx in range(a2-a1):
    for yy in range(b2-b1):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            darea[kk:, xx, yy] = float("nan")

In [6]:
datadir = f'/glade/u/home/{USER}/O2prediction/Prediction/notebooks/data'
filename = '%s/LME-mask-POP_gx1v6.nc' % datadir
ds1 = xr.open_dataset(filename)
lme_mask = ds1['lme_mask'].isel(nlat=range(a1, a2), nlon=range(b1, b2))
lme = [1,2,3,4,5,6,7,8,9,10,65]

In [7]:
for varr in ['O2','AOU','T','S']:
    print(varr)
    filename = f'{path}/DPLE-FOSI_hindcast_Phi_components_{varr}_1948-2017_{tres}.nc'
    ds = xr.open_dataset(filename)
    if tres == 'yearly':
        tnam = 'year'    
    else:
        tnam = 'time'
    nmon = ds.dims[tnam]
    nver = ds.dims['z_t']
    #
    dsf = xr.Dataset()
    for ii in [0,6,12]:
        print(ii)
        dsf[f'Phi_{varr}_{ii}'] = (ds[f'Phi_{varr}_{ii}']*darea).transpose("time","z_t","nlat","nlon")
    # start horizontal averaging within each LME
    phi_box = np.ma.zeros([3, len(lme), nmon, nver])
    phi_box.mask = True
    # select a LME
    for box,nn in zip(lme,range(len(lme))):
        print(box)
        latloc, lonloc = np.where(lme_mask == box)
        ar1 = np.ma.zeros([len(latloc),nver]);ar1.mask=True
        ar2 = np.ma.zeros([3, len(latloc),nmon,nver]);ar2.mask=True
        for bb in range(len(latloc)):
            ar1[bb] = darea.isel(nlat=latloc[bb], nlon=lonloc[bb])
            for ii,xx in zip(range(3),[0,6,12]):
                ar2[ii,bb] = dsf[f'Phi_{varr}_{xx}'].isel(nlat=latloc[bb], nlon=lonloc[bb])
        #
        tarea = np.nansum(ar1,axis=0)
        tarea = np.tile(tarea[np.newaxis,np.newaxis,np.newaxis,:],[3,nmon,1])
        phi_box[:,nn] = np.nansum(ar2,axis=1)/tarea  
    # save data
    dsf = xr.Dataset()
    for ii,xx in zip(range(3),[0,6,12]):
        dsf[f'Phi_{varr}_{xx}'] = xr.DataArray(data=phi_box[ii], coords=dict(lme=('lme', lme), time=('time', ds.coords['time'].values), z_t=('z_t', ds.coords['z_t'].values)), dims=('lme','time','z_t'))
    #
    dsf.load()
    dsf.to_netcdf(f'{path}/DPLE-FOSI_hindcast_Phi_components_{varr}_Eo_space_horizontal_mean_1948-2017_{tres}.nc', mode='w')

O2
0
6
12
1
2
3
4
5
6
7
8
9
10
65
AOU
0
6
12
1
2
3
4
5
6
7
8
9
10
65
T
0
6
12
1
2
3
4
5
6
7
8
9
10
65
S
0
6
12
1
2
3
4
5
6
7
8
9
10
65
